## SimpylCellCounter - An Automated Solution for Quantifying Cfos Cells in Brain Tissue

### Import Modules
#### First, we must import the necessary packages for the processing steps..we also need to load the neural network that classifies overlapping cell objects

In [ ]:
import keras
import h5py
from keras.models import model_from_json
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import math
from collections import deque
import tensorflow as tf
import os
import time

from keras.models import load_model
model = load_model('model_save.h5')


path = 'path\to\image\folder'

save_path = 'path\to\location\to\save\results'

### Set Parameters
#### Now, we must set our parameters for threshold and object size...
#### 1. enter the threshold value into the variable "set_th_value" 
#### 2. set the radius of the smallest cell-to-be counted into the variable "radius"
###### You will notice the variable "circularity_parameter" has a value of 0.70...it is recommended to NOT change this 

In [ ]:
set_th_value = 115 #change this to any desired threshold value
radius = 10 # how large should the counted cells be? Units are in pixels...
circularity_parameter = 0.70 # how circular must our cells be? it is recommended to not alter this value...

### Main Processsing
#### Simply run the following cell for the counting to begin!

In [ ]:
area_parameter = (3.14*radius**2)/0.95 # do NOT change this formula!
area_parameter = area_parameter/1.04 # do NOT change this formula!


dirs = os.listdir(path)
total_counts = []
total_name = []
all_times = []
    
for jj in range(len(dirs)):
#for jj in range(2):
    wdir = dirs[jj]
    fils = os.listdir(path+'\\'+wdir)     
    zor = fils
    #x = fnmatch.filter(fils, "*ofc*")
    def lenfun(e):
        return len(e)    
    Animal = wdir
    x_read = fils
    count_m = [] 
    name_m = []
    time_index = []
    for y in range(len(zor)):
    #for y in range(len(x)):
        #th_value = set_th_value
        t1 = time.time()
        read = path+'\\'+wdir+'\\'+x_read[y]
        #read = path+'\\'+wdir+'\\'+x[y]
        img = cv2.imread(read)[:,:,1]
        #img = cv2.bitwise_not(img)
        th_value=set_th_value
        test_thresh, __ = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        if test_thresh < np.mean(img):
            img = img
        else:
            img = cv2.bitwise_not(img)
        
        new_th, __ = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        if area_parameter > 200:
            if th_value/np.mean(img) > 0.63:
                a = 5
                b = 3
                th_value = th_value
                sig = 'first'   
                if th_value > new_th:
                    a = 5
                    b = 3
                    th_value = new_th
            else:
                a = 3
                b = 1                            
        else:    
            if th_value > new_th:
                a = 3
                b = 1
                th_value = new_th
            else:            
                a = 3
                b = 1
                sig = 'fifth'
                
        ret, img2 = cv2.threshold(img, th_value, 255, cv2.THRESH_BINARY)
        c_function = cv2.medianBlur(img2, 5)
        c_function = cv2.morphologyEx(c_function, cv2.MORPH_CLOSE, np.ones((a,a)), iterations = b)
        d_function = cv2.erode(c_function, np.ones((1,1)), iterations = 1)
        
        
        image, contours, hierarchy = cv2.findContours(d_function, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        if len(contours)==1:
            count_m.append(0)
            continue 
            
        A = []
        df = pd.DataFrame()
        k = np.empty((1,2))
        for j in range(1, len(contours)):
            M = cv2.moments(contours[j])
            areas = M["m00"]
            huMoments = cv2.HuMoments(M)
            if huMoments[1]==0.:
                continue
            if huMoments[2]==0.:
                continue
            if huMoments[3]==0.:
                continue
            if huMoments[4]==0:
                continue
            if huMoments[5]==0:
                continue
            if huMoments[6]==0:
                continue
    
            for i in range(0, len(huMoments)):
                huMoments[i] = -1*math.copysign(1.0, huMoments[i])*math.log10(abs(huMoments[i]))
            huMoments = pd.DataFrame(np.transpose(huMoments))
            df = pd.concat([df, huMoments])
            A.append(areas)

        smallArea = np.where(np.array(A)<area_parameter)
        update_contours = np.delete(np.array(contours), np.asarray(smallArea).astype('int'))
        df = pd.DataFrame(np.delete(np.array(df), np.asarray(smallArea).astype('int'), 0))
        multCirc = np.where(np.array(df)[:,0]<circularity_parameter)[0][1:]

        c = [update_contours[i] for i in multCirc]

        count_array = []
            #mask_create = np.ones((img.shape[0]+100, img.shape[1]+100),np.uint8)
        for h,cnt in enumerate(c):
            #mask = mask_create
            mask = np.ones((img.shape[0]+100, img.shape[1]+100),np.uint8)
            cv2.drawContours(mask,[cnt],0,0,-1)
            #mask = cv2.bitwise_not(mask)
            (x,y,w,h) = cv2.boundingRect(cnt)
            x1 = x-100; x2 = x+100; y1 = y-100; y2 = y+100
            if x1<0:
                x1=0
            if y1<0:
                y1=0
            if x2<0:
                x2=0
            if y2<0:
                y2=0



            mask_subset = mask[y1:y2, x1:x2]
            mask_subset_resize = cv2.resize(mask_subset, (100, 100))
            im4 = tf.keras.utils.normalize(np.array(mask_subset_resize).reshape(-1, 100, 100, 1))
            count_array += [model.predict_classes(im4)]
            
        counts = len(df) + int(sum(np.array(count_array)+1))
            
        
        t2 = time.time()
        
        count_m.append(counts)
        time_index.append(t2-t1)
        #print(th_value, new_th)
        
    count_m = np.asarray(count_m)
    name_m = np.asarray(fils)
    time_index = np.asarray(time_index)
    #name_m = np.asarray(x)    
    total_counts.append(count_m)
    total_name.append(name_m)
    all_times.append(time_index)
    print('Finished counting from '+str(wdir)+'........on to the next one!')
print('All done!')


### Save Results
#### Running this block will save an Excel file of your counts in the location defined by "save_path"

In [ ]:
fn = np.transpose(np.concatenate(total_name, axis=None))
fc = np.transpose(np.concatenate(total_counts, axis=None))
df2 = pd.DataFrame({'image':fn,'counts': fc})
df2.set_index('image')
#df2.to_csv(save_path, index=None)